In [9]:
#https://stackoverflow.com/questions/73071058/fit-data-with-a-lognormal-function-via-maximum-likelihood-estimators/73112103#73112103
#https://towardsdatascience.com/log-normal-distribution-a-simple-explanation-7605864fb67c

In [10]:
from functools import partial
import numpy as np
from scipy import optimize, stats

In [11]:
%store

Stored variables and their in-db values:
arch_capacity_geometry_variables                   -> {'b': 1, 't': 0.5, 'e': 0.08333333333333333}
arch_compressive_demand_thrust                     -> array([525.16363953, 492.96620384, 530.32649903, .
arch_compressive_failure_points_number             -> [69, 69, 69, 69, 259, 1880, 15286, 74760, 99995, 1
compressive_demand_from_arch                       -> {'H_a': array([40.52707914, 33.18183373, 41.704889
compressive_demand_from_backfill                   -> {'H_a': array([246.2211967, 246.2211967, 246.22119
compressive_demand_from_parapet                    -> {'H_a': array([11.18458987,  9.15746235, 11.509639
compressive_demand_from_spandrel_walls             -> {'H_a': array([56.10087926, 45.93299314, 57.731300
compressive_resistance_realisations                -> array([6.07743858, 4.41155859, 6.34456247, ..., 6.
compressive_resistances                            -> [4.691352777703492, 6.2279668592488475]
condition_factor               

In [12]:
%store -r pier_local_scour_depth
%store -r arch_compressive_failure_points_number
%store -r convergence_simulation_number

In [13]:
#Failure probability data from Baker (2015), https://doi.org/10.1193/021113EQS025M
# im = np.array([0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9, 1])
# number_of_analyses = np.array([40, 40, 40, 40, 40, 40, 40, 40])
# number_of_collapses = np.array([0, 0, 0, 4, 6, 13, 12, 16])

#Failure probability data from Qeshta et al., 2021, https://doi.org/10.1080/15732479.2021.1892774
# im = np.array([4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.46])
# number_of_analyses = np.array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]) 
# number_of_collapses = np.array([0, 0, 0, 0, 0, 0, 1, 3, 7, 9, 10])

#Failure probability data from "twelve_arch_compressive_limit_state_equation.ipynb"
# im = np.array([0.289, 0.578, 0.867, 1.156, 1.444, 1.733, 2.022, 2.311, 2.6])
# number_of_analyses = np.array([100_000, 100_000, 100_000, 100_000, 100_000, 100_000, 100_000, 100_000, 100_000])
# number_of_collapses = np.array([69, 69, 69, 259, 1880, 15286, 74760, 99995, 100_000])

#Failure probability data from "twelve_arch_compressive_limit_state_equation.ipynb" and other stored values
im = np.array(pier_local_scour_depth[0])
number_of_analyses = np.ones(len(pier_local_scour_depth[0]))*convergence_simulation_number
number_of_collapses = np.array(arch_compressive_failure_points_number)

In [14]:
np.seterr(divide = 'ignore') #equal to IFERROR function of EXCEL
im_log = np.log(im)

FORMAT_STRING = "{:<20}{:<20}{:<20}"
print(FORMAT_STRING.format("teta", "beta", "log_likelihood_sum"))

teta                beta                log_likelihood_sum  


In [15]:
def neg_log_likelihood_sum(params, im_l, no_a, no_c):
    
    teta, beta = params
    theoretical_fragility_function = stats.norm(np.log(teta), beta).cdf(im_l)
    likelihood = stats.binom.pmf(no_c, no_a, theoretical_fragility_function)
    #https://stackoverflow.com/questions/21610198/runtimewarning-divide-by-zero-encountered-in-log
    log_likelihood = np.log(likelihood)
    log_likelihood_sum = np.sum(log_likelihood)

    print(FORMAT_STRING.format(teta, beta, log_likelihood_sum))
    
    return -log_likelihood_sum

In [16]:
guess_values = (0.5, 0.1) #guess_values = (1.6, 0.1) for Baker and my data, while guess_values = (8, 0.1) for Qeshta et al. (2021) data. 
params_bounds = ((0.01, None), (0.01, None)) #teta and beta must be positive because of the problem physics
desired_tolerance = 1e-6
number_of_iterations = 1000

neg_log_likelihood_sum_partial = partial(neg_log_likelihood_sum, im_l=im_log, no_a=number_of_analyses, no_c=number_of_collapses)

#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
res = optimize.minimize(neg_log_likelihood_sum_partial, x0 = guess_values, 
                method="Nelder-Mead", options={'maxiter':number_of_iterations, 
                'disp':True}, bounds = params_bounds, tol = desired_tolerance)
print(res)

0.5                 0.1                 -inf                
0.525               0.1                 -inf                
0.5                 0.10500000000000001 -inf                
0.5249999999999999  0.095               -inf                
0.50625             0.10250000000000001 -inf                
0.5125              0.1                 -inf                
0.5                 0.10250000000000001 -inf                
0.5125              0.0975              -inf                
0.503125            0.10125             -inf                
0.50625             0.1                 -inf                
0.5                 0.10125             -inf                
0.5062500000000001  0.09875             -inf                
0.5015625           0.100625            -inf                
0.503125            0.1                 -inf                
0.5                 0.100625            -inf                
0.503125            0.099375            -inf                
0.50078125          0.10

c:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_optimize.py:790: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):


0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1                 -inf                
0.5                 0.1 

C:\Users\fn20944\AppData\Local\Temp\ipykernel_9080\3676437843.py:9: RuntimeWarning: Maximum number of iterations has been exceeded.
  res = optimize.minimize(neg_log_likelihood_sum_partial, x0 = guess_values,
